In [14]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Generate numerical features
data = pd.DataFrame({
    'X1': np.random.randint(1, 11, 100),        # Integer values 1-10
    'X2': np.random.normal(50, 15, 100),        # Normal distribution (μ=50, σ=15)
    'X3': np.random.exponential(2, 100),        # Exponential distribution
    'X4': np.random.uniform(0, 100, 100),       # Uniform distribution 0-100
    'X5': np.abs(np.random.randn(100) * 10),    # Right-skewed positive values
    'X6': np.cumsum(np.random.randn(100))       # Time-series like feature
})

# Create target with clear patterns
y = pd.Series(np.where(
    ((data['X1'] > 5) & (data['X2'] < 60)) |    # Rule 1
    ((data['X3'] > 2.5) & (data['X4'] < 40)) |  # Rule 2
    ((data['X5'] > 8) & (data['X6'].diff() > 0)),# Rule 3
    1, 0
))

# Add 15% noise to target
noise_mask = np.random.choice([True, False], 100, p=[0.15, 0.85])
y[noise_mask] = 1 - y[noise_mask]

# Reset index to show 100 entries
data = data.reset_index(drop=True)
y = y.reset_index(drop=True)

# Split Criterion

## Gini Impurity - Classification

Formula: 

$ \text{Gini}(S) = 1 - \sum_{i=1}^{k} p_i^2 $

In [2]:
def gini_impurity(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    gini = 1 - np.sum(probabilities**2)
    return gini

## Entropy - Classification

Formula:

$\text{Entropy}(S) = -\sum_{i=1}^{k} p_i \log_2(p_i) $

In [3]:
def entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))

## Combine split function for classification

In [4]:
def best_split(data, y, split_criterion = 'Gini'):
    min_criterion = float('inf')
    best_feature, best_threshold = None, None

    for feature in data.columns:
        thresholds = data[feature].unique()
        for threshold in thresholds:
            left_mask = data[feature] <= threshold
            right_mask = data[feature] > threshold
            if left_mask.any() and right_mask.any():
                y_left, y_right = y[left_mask], y[right_mask]
                if split_criterion == 'Gini':
                    l_criterion = gini_impurity(y_left)
                    r_criterion = gini_impurity(y_right)
                elif split_criterion == 'Entropy':
                    l_criterion = entropy(y_left)
                    r_criterion = entropy(y_right)
                n_left, n_right = len(y_left), len(y_right)
                weighted_criterion = (n_left / len(y)) * l_criterion + (n_right / len(y)) * r_criterion

                if weighted_criterion < min_criterion:
                    min_criterion = weighted_criterion
                    best_feature = feature
                    best_threshold = threshold

    return best_feature, best_threshold, min_criterion


In [16]:
feature, threshold, gini = best_split(data, y, split_criterion= 'Gini')
print(f"Best split is at feature {feature} <= {threshold} with Gini impurity {gini:.3f}")

feature, threshold, gini = best_split(data, y, split_criterion= 'Entropy')
print(f"Best split is at feature {feature} <= {threshold} with Entropy {gini:.3f}")

Best split is at feature X5 <= 5.39745359324594 with Gini impurity 0.432
Best split is at feature X5 <= 5.39745359324594 with Entropy 0.899


In [12]:
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, leaf_value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.leaf_value = leaf_value

def build_tree(data, y, split_criterion, depth=0, max_depth=None):
    if len(set(y)) == 1 or len(y) == 0:
        # Return a leaf node if all target values are the same or no data is left
        return TreeNode(leaf_value=y.mode()[0] if len(y) != 0 else None)

    if max_depth is not None and depth >= max_depth:
        # Return a leaf node if the maximum depth is reached
        return TreeNode(leaf_value=y.mode()[0])

    # Find the best split
    feature, threshold, _ = best_split(data, y, split_criterion)

    if feature is None:
        # Return a leaf node if no valid split is found
        return TreeNode(leaf_value=y.mode()[0])

    # Split data
    left_mask = data[feature] <= threshold
    right_mask = data[feature] > threshold
    left_data, right_data = data[left_mask], data[right_mask]
    left_y, right_y = y[left_mask], y[right_mask]

    # Recursively build the tree
    left_child = build_tree(left_data, left_y, split_criterion, depth+1, max_depth)
    right_child = build_tree(right_data, right_y, split_criterion, depth+1, max_depth)

    # Return current node
    return TreeNode(feature=feature, threshold=threshold, left=left_child, right=right_child)

# Example usage
tree = build_tree(data, y, split_criterion = 'Gini', max_depth=None)


In [17]:
def display_tree(node, prefix="", is_left=None):
    if node.leaf_value is not None:
        branch = ""
        if is_left is not None:
            branch = "├── " if is_left else "└── "
        print(f"{prefix}{branch}Predict {node.leaf_value}")
        return

    if is_left is None:
        # Root node
        print(f"{node.feature} <= {node.threshold}")
    else:
        branch = "├── " if is_left else "└── "
        print(f"{prefix}{branch}{node.feature} <= {node.threshold}")

    # Calculate new prefix for children
    if is_left is not None:
        new_prefix = prefix + ("│   " if is_left else "    ")
    else:
        new_prefix = prefix + "    "

    if node.left:
        display_tree(node.left, new_prefix, True)
    if node.right:
        display_tree(node.right, new_prefix, False)
        
# Build the tree
tree = build_tree(data, y, split_criterion='Gini', max_depth=None)

# Display the tree
display_tree(tree)


X5 <= 5.39745359324594
    ├── X2 <= 71.01982347040436
    │   ├── X5 <= 4.58441296561979
    │   │   ├── X4 <= 27.15429158197419
    │   │   │   ├── X2 <= 30.33013661373336
    │   │   │   │   ├── Predict 0
    │   │   │   │   └── Predict 1
    │   │   │   └── X4 <= 54.79718832480873
    │   │   │       ├── Predict 0
    │   │   │       └── X6 <= 4.35178960859486
    │   │   │           ├── Predict 0
    │   │   │           └── X3 <= 0.29380380498430403
    │   │   │               ├── Predict 0
    │   │   │               └── X2 <= 59.082246962715125
    │   │   │                   ├── Predict 1
    │   │   │                   └── X1 <= 5
    │   │   │                       ├── Predict 1
    │   │   │                       └── Predict 0
    │   │   └── Predict 0
    │   └── Predict 1
    └── X4 <= 12.76897294224656
        ├── X5 <= 13.723777965472465
        │   ├── Predict 0
        │   └── X6 <= -2.2712967985001415
        │       ├── Predict 0
        │       └── Predict 1
       

In [9]:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, leaf_value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.leaf_value = leaf_value

In [10]:
print(tree.feature)
print(tree.threshold)
print(tree.leaf_value)

X1
2
None


## Variance Reduction - Regression

Formula:

$ \text{Variance}(S) = \frac{1}{|S|} \sum_{i \in S} (y_i - \overline{y}_S)^2 $

$ \text{Variance Reduction} = \text{Var}(S) - \left(\frac{n_{\text{left}}}{n} \times \text{Var}(S_{\text{left}}) + \frac{n_{\text{right}}}{n} \times \text{Var}(S_{\text{right}})\right) $

In [9]:
# Feature
X = np.array([1, 2, 3, 4, 5, 6])
# Regression target
y = np.array([1, 3, 2, 5, 7, 6])

In [10]:
def variance(y):
    return np.var(y)

def best_split_regression(X, y):
    best_var_reduction = -np.inf  # We want to maximize variance reduction
    best_idx, best_thr = None, None
    parent_variance = variance(y)

    for idx in range(len(X)):
        thr = X[idx]
        left_mask = X < thr
        right_mask = X >= thr
        if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
            continue
        left_var = variance(y[left_mask])
        right_var = variance(y[right_mask])
        n_left, n_right = len(y[left_mask]), len(y[right_mask])
        n = len(y)
        # Weighted average variance
        weighted_var = (n_left / n) * left_var + (n_right / n) * right_var
        var_reduction = parent_variance - weighted_var

        if var_reduction > best_var_reduction:
            best_var_reduction = var_reduction
            best_idx, best_thr = idx, thr

    return best_idx, best_thr, best_var_reduction


In [ ]:
index, threshold, var_reduction = best_split_regression(X, y)
print("Best split at index:", index, "with threshold:", threshold, "and Variance Reduction:", var_reduction)

## Build Tree